$$
f = \frac{f_{\max}-f_{\min}}{q_{\max}-q_{\min}} (q-Z)=S(q-Z)\\
q = \frac fS+Z
$$

- unsigned 8 bit
    - [0, 255]（q_max, q_min）
- signed 8 bit
    - [-128, 127]（q_max, q_min)

## quantize

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
model

In [ ]:
weights = model.state_dict()['encoder.layer.0.attention.self.query.weight']
weights.dtype

In [ ]:
# (4h, h)
weights.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
_ = plt.hist(weights.flatten().numpy(), bins=250, )

In [ ]:
# signed 8-bit integer
zero_point = 0
scale = (weights.max() - weights.min())/(127-(-128))
scale

In [ ]:
(weights/scale + zero_point).round().char()

In [ ]:
import torch
from torch import quantize_per_tensor

In [ ]:
q_weights = quantize_per_tensor(weights, scale, zero_point, torch.qint8)
q_weights.int_repr()

In [ ]:
q_weights.shape

## analysis

In [ ]:
_ = plt.hist(q_weights.int_repr().flatten().numpy(), bins=250, range=(-60, 60))

In [ ]:
%%timeit
weights @ weights.T

In [1]:
import torch
from torch.nn.quantized import QFunctional
q_fn = QFunctional()

In [2]:
# qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.backends.quantized.engine

'x86'

In [ ]:
# %%timeit 
q_fn.mul(q_weights, q_weights)